IA & Data science (3i026) -- 2018-2019
--------
*&copy; Equipe pédagogique: Manon Ansart, Vincent Guigue, Marie-Jeanne Lesot, Christophe Marsala, Olivier Schwander.*


# TD-TME03: programmation du perceptron

Dans ce TP, nous allons principalement programmer un perceptron, en nous restreignant au cas binaire où les données sont étiquetées $+1$ ou $-1$ (comme les séances précédentes).

<font size="+1" color="RED">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms :**

Jouve Vincent et Cadiou Antoine

<font color="RED" size="+1">**[Q]**</font> **Renommer ce fichier ipython**

Tout en haut de cette page, cliquer sur <tt>tme-03</tt> et rajouter à la suite de <tt>tme-03</tt> les noms des membres du binômes séparés par un tiret.

<font color="RED" size="+1">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : *tme-03-Nom1_Nom2.ipynb* 
- *Nom1* et *Nom2* : noms des membres du binôme
- ne pas compresser ou faire une archive: il faut rendre le fichier ipython tel quel, éventuellement, si vous avez d'autres fichiers vous les rendez séparément.

**Echancier pour la soumission de votre compte-rendu:**
- le compte-rendu d'une séance doit être remis obligatoirement <font color="RED">** à la fin de la séance**</font>. C'est ce compte-rendu qui donne la note de base de la séance.
- vous pouvez éventuellement compléter votre compte-rendu  pour obtenir des points bonus, dans ce cas, vous devez soumettre votre complément avant le début de la semaine suivante.

** Sur la page Moodle de remise du travail <font color="RED">ne pas oublier d'envoyer le compte rendu</font>** à la fin de la séance, la soumission de la version complémentaire post-séance se fera sur une page différente.

## Création d'une librairie

Afin de pouvoir réutiliser les classes et fonctions écrites dans les séances de TDTME précédentes, on va construire une librairie qu'il suffira d'importer.

Récupérer l'archive iads.tgz et la désarchiver de telle sorte que le répertoire iads soit un répertoire frère du répertoire tme03.

Ainsi, vous devrez avoir l'arborescence suivante dans votre répertoire $HOME:

    - 3i026/
        - tme01/
            - tme01.ipynb
        - tme02/
            - tme02.ipynb
        - tme03/
            - tme03.ipynb
        - iads/
            - Classifiers.py
            - LabeledSet.py
            - utils.py
          

**Important** :
- ce fichier tme03.ipynb doit toujours rester dans le répertorie tme03/
- pour ouvrir les fichiers py qui se trouvent dans le répertoire iads/ il est nécessaire d'utiliser un éditeur de texte comme emacs, gedit, idle,...


<font color="RED" size="+1">**[Q]**</font> **Ouvrir et compléter les fichiers Classifiers.py et utils.py**

Pour compléter ces fichiers, reprendre le code écrit dans le TDTME précédent pour compléter toute les parties indiquées <tt>#TODO</tt>.
- dans utils.py, il faut compléter la fonction createGaussianDataset
- dans Classifiers.py, il faut compléter le code des classifiers

Une fois ces fichiers complétés, la librairie <tt>iads</tt> peut être importée dans ce notebook.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

# La ligne suivante permet de préciser le chemin d'accès à la librairie iads
import sys
sys.path.append('../')

# Importation de la librairie iads
import iads as iads

# importation de LabeledSet
from iads import LabeledSet as ls

# importation de Classifiers
from iads import Classifiers as cl

# importation de utils
# from iads import utils_iads as ut

ModuleNotFoundError: No module named 'iads'

In [ ]:
import random

In [ ]:
help(ut)

La commande <tt>help</tt> permet d'avoir des informations sur le contenu d'une librairie importée:

In [ ]:
help(iads)

In [ ]:
help(ls)

In [ ]:
help(cl)

Dans les boîtes suivantes, on utilise ces librairies avec les exemples vus en TDTME-02.

In [ ]:
# Exemple d'utilisation de LabeledSet

une_base = ls.LabeledSet(2)        # définition d'une base pour contenir des exemples en 2D

une_base.addExample([0, 1],1)   # ajout de l'exemple (0, 1) de classe +1
une_base.addExample([2, 3],1)   # ajout de l'exemple (2, 3) de classe +1
une_base.addExample([1, 2],-1)  # ajout de l'exemple (1, 2) de classe -1
une_base.addExample([2, 2],-1)  # ajout de l'exemple (2, 2) de classe -1

# Fonction pour afficher le LabeledSet
def affiche_base(LS):
    """ LabeledSet
        affiche le contenu de LS
    """
    for i in range(0,LS.size()):
        print("Exemple "+str(i))
        print("\tdescription : ",LS.getX(i))
        print("\tlabel : ",LS.getY(i))
    return

# Affichage de la base
affiche_base(une_base)

In [ ]:
def plot2DSet(set):
    """ LabeledSet -> NoneType
        Hypothèse: set est de dimension 2
        affiche une représentation graphique du LabeledSet
        remarque: l'ordre des labels dans set peut être quelconque
    """
    S_pos = set.x[np.where(set.y == 1),:][0]      # tous les exemples de label +1
    S_neg = set.x[np.where(set.y == -1),:][0]     # tous les exemples de label -1
    plt.scatter(S_pos[:,0],S_pos[:,1],marker='o') # 'o' pour la classe +1
    plt.scatter(S_neg[:,0],S_neg[:,1],marker='x') # 'x' pour la classe -1

def plot_frontiere(set,classifier,step=10):
    """ LabeledSet * Classifier * int -> NoneType
        Remarque: le 3e argument est optionnel et donne la "résolution" du tracé
        affiche la frontière de décision associée au classifieur
    """
    mmax=set.x.max(0)
    mmin=set.x.min(0)
    x1grid,x2grid=np.meshgrid(np.linspace(mmin[0],mmax[0],step),np.linspace(mmin[1],mmax[1],step))
    grid=np.hstack((x1grid.reshape(x1grid.size,1),x2grid.reshape(x2grid.size,1)))
    
    # calcul de la prediction pour chaque point de la grille
    res=np.array([classifier.predict(grid[i,:]) for i in range(len(grid)) ])
    res=res.reshape(x1grid.shape)
    # tracer des frontieres
    plt.contourf(x1grid,x2grid,res,colors=["red","cyan"],levels=[-1000,0,1000])
    
# ------------------------ 

def createGaussianDataset(positive_center, positive_sigma, negative_center, negative_sigma, nb_points):
    """ 
        rend un LabeledSet 2D généré aléatoirement.
        Arguments:
        - positive_center (vecteur taille 2): centre de la gaussienne des points positifs
        - positive_sigma (matrice 2*2): variance de la gaussienne des points positifs
        - negative_center (vecteur taille 2): centre de la gaussienne des points négative
        - negative_sigma (matrice 2*2): variance de la gaussienne des points négative
        - nb_points (int):  nombre de points de chaque classe à générer
    """
    labeledSet = ls.LabeledSet(2)
    for i in range(0, nb_points):
        pos = np.random.multivariate_normal(positive_center,positive_sigma)
        neg = np.random.multivariate_normal(negative_center,negative_sigma)
        labeledSet.addExample(pos,1)
        labeledSet.addExample(neg,-1)
    return labeledSet

In [ ]:
# Exemple d'utilisation de utils

the_set = createGaussianDataset(np.array([1,1]),np.array([[1,0],[0,1]]),np.array([-1,-1]),np.array([[1,0],[0,1]]),10)
the_set100 = createGaussianDataset(np.array([1,1]),np.array([[1,0],[0,1]]),np.array([-1,-1]),np.array([[1,0],[0,1]]),100)

print("Taille de la base jouet générée :", the_set.size(), "exemples")

# Affichage :
plot2DSet(the_set)


In [ ]:
# Exemple d'utilisation de Classifiers pour créer un knn:

knn = cl.ClassifierKNN(2,3)

knn.train(the_set)

plot_frontiere(the_set,knn)
plot2DSet(the_set)

<font color="RED" size="+2">**Très important**</font>: tout le reste du travail à faire dans ce TDTME ne doit être fait que dans le notebook. Il ne faut plus modifier les fichiers de la librairie.

Le compte-rendu de cette séance ne comportera que le fichier notebook.


Perceptron aléatoire
====

La classe <tt>ClassifierPerceptronRandom</tt> suivante hérite de la classe `Classifier`. Elle implémente un modèle de perceptron aléatoire qui modélise un hyperplan $f_w(x)=\langle w ; x \rangle$ tel que $w$ est tiré aléatoirement lors de la création.





In [ ]:
class ClassifierPerceptronRandom(cl.Classifier):
    def __init__(self, input_dimension):
        """ Argument:
                - input_dimension (int) : dimension d'entrée des exemples
            Hypothèse : input_dimension > 0
        """
        v = np.random.rand(input_dimension)     # vecteur aléatoire à input_dimension dimensions
        self.w = (2* v - 1) / np.linalg.norm(v) # on normalise par la norme de v

    def predict(self, x):
        """ rend la prediction sur x (-1 ou +1)
        """
        z = np.dot(x, self.w)
        return z
        
    def train(self,labeledSet):
        """ Permet d'entrainer le modele sur l'ensemble donné
        """        
        print("No training needed")

<font color="RED" size="+1">**[Q]**</font> Tester ce modèle de perceptron sur un jeu de données aléatoire.


In [ ]:
# Exemple de création et d'utilisation d'un Perceptron aléatoire:

perceptron_random = ClassifierPerceptronRandom(2)

plot_frontiere(the_set,perceptron_random)
plot2DSet(the_set)

Perceptron de Rosenblatt
===

<font color="RED" size="+1">**[Q]**</font> En héritant de la classe `Classifier`, écrire la classe <tt>ClassifierPerceptron</tt> qui implémente l'algorithme du Perceptron de Rosenblatt vu en cours.

La fonction `train` fera une itération sur l'ensemble des données de l'apprentissage. Les points d'apprentissage seront tirés dans un ordre aléatoire.



In [ ]:
class ClassifierPerceptron(cl.Classifier):
    """ Perceptron de Rosenblatt
    """
    def __init__(self,input_dimension,learning_rate):
        """ Argument:
                - intput_dimension (int) : dimension d'entrée des exemples
                - learning_rate :
            Hypothèse : input_dimension > 0
        """
        self.dim = input_dimension
        self.learning_rate = learning_rate
        self.w = np.array([0]*self.dim)

    def predict(self,x):
        """ rend la prediction sur x (-1 ou +1)
        """
        z = np.dot(x, self.w)
        return z

    
    def train(self,labeledSet):
        """ Permet d'entrainer le modele sur l'ensemble donné
        """
        tab = ([k for k in range(labeledSet.size())])
        random.shuffle(tab)
        for i in tab:
            x = labeledSet.getX(i)
            y = labeledSet.getY(i)
            if (self.predict(x)*y <= 0):
                self.w = self.w + self.learning_rate*(x * y)

In [ ]:
perceptron = ClassifierPerceptron(2, 0.1)
perceptron.train(the_set)

plot_frontiere(the_set, perceptron)
plot2DSet(the_set)

<font color="RED" size="+1">**[Q]**</font> Tester le perceptron de la manière suivante:
- Apprendre pendant N itérations
- Afficher l'accuracy du modèle à chaque itération
- Vérifier que l'accuracy baisse
- Dessiner la frontière obtenue

In [ ]:
N = 10
perceptron = ClassifierPerceptron(2, 1)
accuracies=[]
for i in range (N):
    perceptron.train(the_set)
    accuracies.append(perceptron.accuracy(the_set))
plt.ylim(0.5,1)
plt.plot(accuracies)
plt.show()

In [ ]:
N = 10
perceptron = ClassifierPerceptron(2, 0.1)
accuracies=[]
for i in range (N):
    perceptron.train(the_set100)
    accuracies.append(perceptron.accuracy(the_set100))
plt.ylim(0.5,1)
plt.plot(accuracies)
plt.show()

In [ ]:
N = 100
for epsilon in [1/(10**n) for n in range(1, 5)]:
    perceptron = ClassifierPerceptron(2, epsilon)
    accuracies=[]
    for i in range (N):
        perceptron.train(the_set100)
        accuracies.append(perceptron.accuracy(the_set100))
    print("epsilon=",epsilon)
    plt.plot(accuracies)
    plt.show()

Courbe 1 : On observe qu'il y a beaucuoup de variations de la précision du classifieur, c'est dû au taux d'apprentissage qui est trop grand (donc w varie trop brutalement).

Courbe 2: même observation que la courbe 1 mais il y a moins d'amplitude dans les précisions

Courbe 3: on a convergé vers une précision constante, ce qui fait que ce taux d'apprentissage, pour ce jeu de données et le nombre de fois où on a appris (N) est cohérent

Courbe 4: Le taux d'apprentissage est trop petit donc w varie trop lentement pour notre nombre (N) d'apprentissage

<font color="RED" size="+1">**[Q]**</font> Expérimentations

- Générer deux ensembles de train et de test
- Dessiner la courbe de performance en fonction de l'itération sur les ensembles de train et de test
    * Que se passe-t-il en fonction du pas (epsilon) d'apprentissage ? 
    * Que se passe-t-il quand le nombre d'exemples en apprentissage est faible ? 


In [ ]:
train = createGaussianDataset(np.array([1,1]),np.array([[1,0],[0,1]]),np.array([-1,-1]),np.array([[1,0],[0,1]]),100)
test = createGaussianDataset(np.array([1,1]),np.array([[1,0],[0,1]]),np.array([-1,-1]),np.array([[1,0],[0,1]]),100)

N = 100
for epsilon in [1/(10**n) for n in range(1, 5)]:
    perceptron = ClassifierPerceptron(2, epsilon)
    accuracies=[]
    for i in range (N):
        perceptron.train(train)
        accuracies.append(perceptron.accuracy(test))
    print("epsilon=",epsilon)
    plt.plot(accuracies)
    plt.show()


<font color="RED" size="+1">**[Q]**</font> **Données XOR**

Ecrire la fpnction `createXOR` qui, étant donné un nombre de points $n$ et une variance $var$, permet de générer une base de données contenant:
- deux gaussiennes centrées en (0,0) et (1,1) correspondant à $n$ exemples positifs
- deux gaussiennes centrées en (1,0) et (0,1) correspondant à $n$ exemples négatifs

Dans la suite, on appelle XOR une base de données créée par cette fonction.


Que se passe-t-il quand on apprend un perceptron  ? Quel est le problème ? 

In [ ]:
def createXOR(n, var):
    labeledSet = ls.LabeledSet(2)
    for i in range(0, n):
        labeledSet.addExample(np.random.multivariate_normal([0,0],[[var,0],[0,var]]),1)
        labeledSet.addExample(np.random.multivariate_normal([1,1],[[var,0],[0,var]]),1)
        labeledSet.addExample(np.random.multivariate_normal([1,0],[[var,0],[0,var]]),-1)
        labeledSet.addExample(np.random.multivariate_normal([0,1],[[var,0],[0,var]]),-1)
    return labeledSet

In [ ]:
xor=createXOR(100,0.01)
plot2DSet(xor)

In [ ]:
N = 100
perceptronXor = ClassifierPerceptron(2, 0.001)
accuracies=[]
for i in range (N):
    perceptronXor.train(xor)
    accuracies.append(perceptronXor.accuracy(xor))
plt.ylim(0,1)
plt.plot(accuracies)
plt.show()

on observe que la précision tourne autour de 0.5 ce qui est normal car on ne peut pas tracer une droite qui passe par l'origine est qui sépare bien nos classe de données

Kernel Trick
--------------

Nous allons projeter (manuellement) les données 2D dans un espace de plus grande dimension. Voici un exemple de projection qui transforme un vecteur $(x_1,x_2)$ en un vecteur $(x_1,x_2,1)$

In [ ]:
class KernelBias:
    def transform(self,x):
        y=np.asarray([x[0],x[1],1])
        return y

k=KernelBias()
k.transform(the_set.getX(1))


<font color="RED" size="+1">**[Q]**</font> Implémenter la classe ClassifierPerceptronKernel qui prend un Kernel en paramètre, et calcule le perceptron sur la version "kernélisée" des données. Tester ce perceptron sur le dataset 1 (2 gaussiennes) et le dataset XOR.

In [2]:
class ClassifierPerceptronKernel(cl.Classifier):
    def __init__(self,dimension_kernel,learning_rate,kernel):
        """ Argument:
                - intput_dimension (int) : dimension d'entrée des exemples
                - learning_rate :
            Hypothèse : input_dimension > 0
        """
        self.dim = dimension_kernel
        self.learning_rate = learning_rate
        self.kernel=kernel
        self.w = np.array([0]*self.dim)

    def predict(self,x):
        """ rend la prediction sur x (-1 ou +1)
        """
        xtrans = self.kernel.transform(x)
        z = np.dot(xtrans, self.w)
        return z

    
    def train(self,labeledSet):
        """ Permet d'entrainer le modele sur l'ensemble donné
        """
        tab = ([k for k in range(labeledSet.size())])
        random.shuffle(tab)
        for i in tab:
            x = labeledSet.getX(i)
            xtrans = self.kernel.transform(x)
            y = labeledSet.getY(i)
            if (self.predict(x)*y <= 0):
                self.w = self.w + self.learning_rate*(xtrans * y)

NameError: name 'cl' is not defined

In [ ]:
N = 100
setKernel=createGaussianDataset(np.array([2,2]),np.array([[1,0],[0,1]]),np.array([4,4]),np.array([[1,0],[0,1]]),100)
perceptronKernel = ClassifierPerceptronKernel(3, 0.001, k)
for i in range (N):
    perceptronKernel.train(setKernel)
plot_frontiere(setKernel, perceptronKernel)
plot2DSet(setKernel)

In [ ]:
N = 100
perceptronXorKernel = ClassifierPerceptronKernel(3, 0.01,k)
for i in range (N):
    perceptronXorKernel.train(xor)
plot_frontiere(xor, perceptronXorKernel)
plot2DSet(xor)

<font color="RED" size="+1">**[Q]**</font> Implémenter le kernel : $(x_1,x_2) \rightarrow (1,x_1,x_2,x_1*x_1,x_2*x_2,x_1*x_2)$. Entrainer le perceptron correspondant sur le XOR. Que constatez-vous ? Donnez une explication (explication donnée en cours lors de la prochaine séance)

In [ ]:
class KernelPoly:
    def transform(self,x):
        y=np.asarray([1, x[0], x[1], x[0]*x[0], x[1]*x[1], x[0]*x[1]])
        return y

N=100
k2=KernelPoly()
perceptron_k = ClassifierPerceptronKernel(6, 0.001, k2)

for i in range (N):
    perceptron_k.train(xor)
plot_frontiere(xor, perceptron_k)
plot2DSet(xor)


Expérimentations sur des datasets réels
-------



A partir de datasets trouvés sur le web, réaliser un ensemble d'expérimentations permettant de comparer les 3 classifiers (knn, perceptron de Rosenblatt et version kernélisée) que vous avez implémentés.

Par exemple, vous pouvez utiliser les datasets suivants:
- https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Coimbra
- https://www.kaggle.com/uciml/indian-liver-patient-records
- voir sur les données ouvertes de Paris : https://opendata.paris.fr/explore/?sort=modified

Afin d'utiliser de tels jeux de données, il sera certainement nécessaire que vous réalisiez un prétraitement afin de pouvoir utiliser vos implémentations.